In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    api_key='',
    max_tokens=512,
    timeout=None,
    max_retries=2,
    # other params...
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

According to NASA's Apollo 11 mission, the first person to set foot on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, at 2:56 UTC. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2848228/software-development-engineer?cmpid=SPLICX0248M&utm_source=linkedin.com&utm_campaign=cxro&utm_medium=social_media&utm_content=job_posting&ss=paid")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Software Development Engineer - Job ID: 2848228 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsDiversity, equity, and inclusionInterview tipsLeadership principlesWorking at AmazonFAQ×Software Development EngineerJob ID: 2848228 | Amazon.com Services LLCApply nowDESCRIPTIONDo you enjoy solving complex problems with simple solutions? Would you like to help us innovate on behalf of Amazon’s Digital and Device customers?Device, Digital and Alexa Support (D2AS) technology organization is seeking an experienced software development engineer to join the engineering team. The D2AS vision is to create innovative support experiences for customers of Amazon’s devices and digital products, through intelligent self-service solutions and enabling our tech-support organization with tools and technology. The job will give you direct experience building software included on the

In [3]:
from langchain_core.prompts import PromptTemplate
import json
import re

# Define the prompt template
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: `role`, `experience`, `skills`, and `description`.
    Return only the JSON, without any additional text, explanations, or preamble.
    ### VALID JSON (NO PREAMBLE):    
    """
)

# Execute the chain
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data': page_data})

# Raw response from the LLM
raw_response = res.content

# Use regex to extract valid JSON
match = re.search(r'\{.*\}', raw_response, re.DOTALL)
if match:
    json_part = match.group()  # Extract the JSON part
    try:
        extracted_json = json.loads(json_part)  # Parse JSON
        print(json.dumps(extracted_json, indent=4))  # Pretty-print the JSON
    except json.JSONDecodeError as e:
        print("Error: The extracted text is not valid JSON.", e)
else:
    print("Error: No valid JSON found in the response.")

{
    "role": "Software Development Engineer",
    "experience": {
        "min": 3,
        "max": null,
        "type": "years",
        "description": "non-internship professional software development experience"
    },
    "skills": {
        "programming languages": [
            "at least one software programming language"
        ],
        "design": [
            "design patterns",
            "reliability and scaling"
        ],
        "full software development life cycle": [
            "coding standards",
            "code reviews",
            "source control management",
            "build processes",
            "testing",
            "operations"
        ]
    },
    "description": "Do you enjoy solving complex problems with simple solutions? Would you like to help us innovate on behalf of Amazon\u2019s Digital and Device customers? Device, Digital and Alexa Support (D2AS) technology organization is seeking an experienced software development engineer to join the engin

In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(raw_response)
json_res

{'role': 'Software Development Engineer',
 'experience': {'min': 3,
  'max': None,
  'type': 'years',
  'description': 'non-internship professional software development experience'},
 'skills': {'programming languages': ['at least one software programming language'],
  'design': ['design patterns', 'reliability and scaling'],
  'full software development life cycle': ['coding standards',
   'code reviews',
   'source control management',
   'build processes',
   'testing',
   'operations']},
 'description': 'Do you enjoy solving complex problems with simple solutions? Would you like to help us innovate on behalf of Amazon’s Digital and Device customers? Device, Digital and Alexa Support (D2AS) technology organization is seeking an experienced software development engineer to join the engineering team.'}

In [5]:
import uuid
import chromadb
import json
from sentence_transformers import SentenceTransformer

# Initialize ChromaDB persistent client
client = chromadb.PersistentClient(path="vectorstore")

# Create or get a collection
collection = client.get_or_create_collection(name="resume")

# Define your resume sections
resume_sections = {
    
    "Technical Skills": """Programming Languages: C, C#, Java, Python, HTML5, CSS, JavaScript, TypeScript, JSON
Frameworks & Technologies: .NET (WinForms, WPF with MVVM, ASP.NET MVC Core), Entity Framework, ADO.NET, Angular, React, Node.js, Razor, Bootstrap v5, Dependency Injection, Swagger, SSIS, SSRS, PowerShell, Visual Studio Extensibility, Spring Boot, Agile/Scrum, CI/CD
Databases & Machine Learning: SQL, PL/SQL, Microsoft SQL Server, MongoDB, PostgreSQL, Scikit-learn, Pandas
Cloud Platforms: Azure, AWS (S3, Lambda, EC2, RDS), GCP
Tools & Version Control: Jenkins, JIRA, TortoiseSVN, Git, Bitbucket, Postman, Visual Studio, IntelliJ, Harness, OpenShift
Certifications: Microsoft Certified: Azure Fundamenta""",
    
    "Professional Experience": """Student Assistant, REGARDS Study, UAB, Birmingham, Alabama August 2023 – April 2024
Developed a patient records management system in ASP.NET Core Web App, enhancing data accuracy and reliability by 30% through real-time updates based on patient profiles, medical histories, survey responses, and real-time health monitoring data.
Designed reusable UI components (Partial Views, Layouts, Razor CRUD), reducing development time by 30% and improving platform-wide UI consistency.
Architected Data Access Layers using ADO.NET and Entity Framework, reducing data retrieval times by 35%. Managed backend services (RESTful, SOAP, WCF and Web APIs), boosting data exchange efficiency by 30%.
Software Engineer – UNISYS India Pvt Ltd., India April 2021 – December 2022
Constructed Web APIs with ASP.NET Core 6.0 for a banking alert system, processing over 1 million alerts daily via OpenShift, ensuring seamless customer notifications.
Optimized Microsoft SQL Server with efficient Data Access Layers (ADO.NET, Entity Framework), reducing query times by 40% for better database performance.
Engineered React JS components (JSX, props, hooks), boosting user engagement by 30% and improving load times by 25% through an interactive user interface.
Utilized Swagger and Postman for API documentation and testing, reducing API error rates by 20%.
Enhanced a custom domain-driven microservices architecture by implementing CQRS and MediatR design patterns, resulting in a 30% improvement in system scalability and maintainability.
Worked in Agile/Scrum using JIRA for project management and GitHub for code management. Implemented GitHub Actions and Harness in the CI/CD pipeline, boosting deployment efficiency by 35% on OpenShift.
Associate Software Engineer – UNISYS India Pvt Ltd., India June 2019 – April 2021
Augmented MCP Project with Visual Studio extensibility features, improving user experience by 75%.
Designed WPF, Custom Controls, User Controls and WinForms using MVVM patterns, boosting UI responsiveness by 30% and user satisfaction by 20%.
Led the development and deployment of a custom debug engine integrated with the MCP Program, revolutionizing debugging capabilities for specialized languages such as Algol and Cobol files.
Contributed significantly (50%) to the Azure Network Watcher research group for MCP, specializing in enhancing network health for MCP's IaaS products, including Virtual Machines, Servers, Virtual Networks, Application Gateways, and Load Balancers.
Created continuous integration and delivery pipelines using Jenkins, reducing deployment time by 40% and increasing deployment frequency by 30%. Managed diverse Agile/Scrum assignments, resolved 200+ bugs, and employed Tortoise SVN for version control.""",
    
    "Project Work": """Event Management App: Web app for customers to explore event services; used React, NodeJS, MongoDB.
IPL Result Prediction: Predicted IPL tournament results with data models; used Python, Flask, Pandas.
Human Activity Mining: Analyzed routines to provide health care suggestions; achieved 92% accuracy using RNN."""
}

# Add sections to the collection with embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Embed the resume sections and add them to the collection
for section, content in resume_sections.items():
    section_embedding = embedder.encode([content])
    collection.add(
        documents=[content],
        metadatas=[{"section": section}],
        embeddings=section_embedding,
        ids=[str(uuid.uuid4())]
    )

# Confirm successful storage
print(f"Collection now has {collection.count()} items.")

Collection now has 22 items.


In [8]:
links = collection.query(query_texts=str(json_res)).get('metadatas', [])
links

[[{'section': 'Professional Experience'},
  {'section': 'Professional Experience'},
  {'section': 'Professional Experience'},
  {'section': 'Technical Skills'},
  {'section': 'Technical Skills'},
  {'section': 'Technical Skills'},
  {'section': 'Education'},
  {'section': 'Education'},
  {'section': 'Education'},
  {'section': 'Education'}]]

In [6]:
# Your job description as a JSON object
job_description_json = json_res

# Convert job description to text (you can modify the structure to match your query requirements)
job_description_text = json.dumps(job_description_json)

# Generate embeddings for the job description
job_description_embedding = embedder.encode([job_description_text])

# Query the collection to find matching sections based on the job description
search_results = collection.query(
    query_embeddings=job_description_embedding,  # Pass the embeddings of the job description
    n_results=5  # Fetch top 5 results
)

# Extract the relevant "links" from the matching results
matching_links = []
for result in search_results['documents']:
    matching_links.append(result)

# Display the matching links based on the job description
print("Matching links based on the job description:")
for link in matching_links:
    print(link)


Matching links based on the job description:
['Student Assistant: Developed a patient records management system improving data accuracy by 30%.\nSoftware Engineer: Built banking alert systems processing over 1M alerts daily; optimized databases for 40% faster query performance.\nAssociate Software Engineer: Improved debugging for specialized languages; enhanced MCP with Azure Network Watcher.', 'Student Assistant: Developed a patient records management system improving data accuracy by 30%.\nSoftware Engineer: Built banking alert systems processing over 1M alerts daily; optimized databases for 40% faster query performance.\nAssociate Software Engineer: Improved debugging for specialized languages; enhanced MCP with Azure Network Watcher.', 'Student Assistant: Developed a patient records management system improving data accuracy by 30%.\nSoftware Engineer: Built banking alert systems processing over 1M alerts daily; optimized databases for 40% faster query performance.\nAssociate Softwa

In [7]:
from langchain.prompts import PromptTemplate

# Generate the email template with an updated prompt that aligns job description with resume
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Phanendra Sai Sree Vinay Alapaka, a software engineer with extensive experience in developing scalable systems, building REST APIs, and working on both frontend and backend technologies. Your job is to write a cold email to the client regarding the job mentioned above, describing how your skills align with their requirements. You should also mention your technical experience in various domains such as cloud, databases, UI development, and backend systems.
    
    Remember you are Phanendra Sai Sree Vinay Alapaka. 
    Do not provide a preamble.

    ### EMAIL (NO PREAMBLE):
    """
)

# Assuming 'llm' is a language model for generating the email
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": job_description_text, "link_list": matching_links})
print(res.content)


Subject: Application for Software Development Engineer Role at Amazon

Dear Hiring Manager,

I came across the job posting for a Software Development Engineer at Amazon's Device, Digital and Alexa Support (D2AS) technology organization and was excited to see that my skills and experience align with the requirements. With over 3 years of non-internship professional software development experience, I am confident that I can make a valuable contribution to your team.

As a seasoned software engineer, I have developed scalable systems, built REST APIs, and worked on both frontend and backend technologies. My technical experience spans various domains, including cloud (AWS), databases (MySQL, MongoDB), UI development (React, Angular), and backend systems (Node.js, Java). I am well-versed in design patterns, reliability, and scaling, and have experience with full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, an